# Representation Learning - Part 3 - t-SNE Animation

We will now use the weights stored from the training routine v5a to generate feature vectors for each wet of weights stored. There will be used to create an animation showing how the model representation changes over the course of training.

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization


# This time we will only use the test set:
_, (x_test, y_test) = cifar10.load_data()
x_test = x_test.astype('float32')
x_test /= 255
print(x_test.shape[0], 'test samples')

y_test = np.asarray(y_test)
print(y_test.shape[0], 'test labels')

Using TensorFlow backend.


10000 test samples


In [2]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_test.shape[1:], name='conv1'))
model.add(BatchNormalization(axis=3, name='bn_conv1'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), name='conv2'))
model.add(BatchNormalization(axis=3, name='bn_conv2'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', name='conv3'))
model.add(BatchNormalization(axis=3, name='bn_conv3'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), name='conv4'))
model.add(BatchNormalization(axis=3, name='bn_conv4'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, name='fc1'))
model.add(BatchNormalization(axis=1, name='bn_fc1'))
model.add(Activation('relu'))
model.add(Dense(num_classes, name='output'))
model.add(BatchNormalization(axis=1, name='bn_outptut'))
model.add(Activation('softmax'))

feat_extractor = Model(inputs=model.input, outputs=model.get_layer('fc1').output)

In [4]:
import os
import numpy as np

out_dir = 'v5a-features'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

weights_paths = [f for f in os.listdir('.') if f.startswith('v5a-weights')]

for weights_path in weights_paths:
    model.load_weights(weights_path)
    features = feat_extractor.predict(x_test, batch_size=32)

    np.save(os.path.join(out_dir, os.path.splitext(weights_path)[0] + 'fc1_features.npy'), features)

## t-SNE with Default Settings - FC 1

In [5]:
from sklearn.manifold import TSNE

for weights_path in weights_paths:
    features = np.load(os.path.join(out_dir, os.path.splitext(weights_path)[0] + 'fc1_features.npy'))
    tsne = TSNE().fit_transform(features)
    np.save(os.path.join(out_dir, os.path.splitext(weights_path)[0] + 'fc1_features_tsne.npy'), tsne)

## Store FC 1 t-SNE Visualizations

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import cifar10

Using TensorFlow backend.


In [2]:
def normalize(tsne):
    tx, ty = tsne[:,0], tsne[:,1]
    tx = (tx-np.min(tx)) / (np.max(tx) - np.min(tx))
    ty = (ty-np.min(ty)) / (np.max(ty) - np.min(ty))
    return tx, ty

In [3]:
weights_paths = [f for f in os.listdir('.') if f.startswith('v5a-weights')]
out_dir = 'v5a-features'

filenames = [os.path.join(out_dir, os.path.splitext(weights_path)[0] + 'fc1_features_tsne.npy')
             for weights_path in weights_paths]

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# This time we will only use the test set:
_, (x_test, y_test) = cifar10.load_data()
y_test = np.asarray(y_test)

In [56]:
plt.close()

for filename in filenames:
    fig = plt.figure(figsize = (16,12))
    ax = fig.add_subplot(111)

    tsne = np.load(filename)
    tx, ty = normalize(tsne)

    for i in range(len(classes)):
        y_i = y_test == i
        plt.scatter(tx[y_i[:, 0]], ty[y_i[:, 0]], label=classes[i])
    epoch_num = int(os.path.split(filename)[1][12:14])
    label = 'Epoch {0}'.format(epoch_num)
    ax.set_title(label)
    plt.legend(loc=4)
    plt.gca().invert_yaxis()
    plt.savefig(os.path.splitext(filename)[0] + '.jpg', bbox_inches='tight')

/home/phil/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


## Plot t-SNE Points as Animation

In [25]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

Check that we have the 'imagemagick' writer in order to save plots in GIF format.

In [26]:
import matplotlib.animation as animation
animation.writers['imagemagick']

matplotlib.animation.ImageMagickWriter

In [52]:
fig = plt.figure(figsize = (8, 6))
ax = plt.axes(xlim=(-0.1, 1.1), ylim=(-0.1, 1.1))
line, = ax.plot([], [], 'ro')
#fig.gca().invert_yaxis()

filenames_sorted = sorted(filenames)

def init():
    line.set_data([], [])
    return line,

def update(i):
    print(filenames_sorted[i])    
    tsne = np.load(filenames_sorted[i])
    tx, ty = normalize(tsne)
    line.set_data(tx, ty)
    
    epoch_num = int(os.path.split(filenames_sorted[i])[1][12:14])
    label = 'Epoch {0}'.format(epoch_num)
    ax.set_xlabel(label)
    return line,

anim = animation.FuncAnimation(fig, update, init_func=init,
                               frames=17, interval=100, blit=True)

In [53]:
anim.save('t-sne.gif', dpi=72, writer='imagemagick', fps=5)

v5a-features/v5a-weights.00-1.2060fc1_features_tsne.npy
v5a-features/v5a-weights.01-0.9069fc1_features_tsne.npy
v5a-features/v5a-weights.02-0.8789fc1_features_tsne.npy
v5a-features/v5a-weights.03-0.7062fc1_features_tsne.npy
v5a-features/v5a-weights.04-0.6751fc1_features_tsne.npy
v5a-features/v5a-weights.05-0.6216fc1_features_tsne.npy
v5a-features/v5a-weights.07-0.6170fc1_features_tsne.npy
v5a-features/v5a-weights.08-0.5633fc1_features_tsne.npy
v5a-features/v5a-weights.10-0.5433fc1_features_tsne.npy
v5a-features/v5a-weights.11-0.4860fc1_features_tsne.npy
v5a-features/v5a-weights.17-0.4610fc1_features_tsne.npy
v5a-features/v5a-weights.20-0.4330fc1_features_tsne.npy
v5a-features/v5a-weights.29-0.4031fc1_features_tsne.npy
v5a-features/v5a-weights.36-0.4006fc1_features_tsne.npy
v5a-features/v5a-weights.41-0.3978fc1_features_tsne.npy
v5a-features/v5a-weights.48-0.3949fc1_features_tsne.npy
v5a-features/v5a-weights.59-0.3879fc1_features_tsne.npy


In [41]:
from IPython.display import HTML, Image

Image(url='t-sne.gif')